# Tutorial 2, Part 3 - Site Effects in ABIDE

In [1]:
%Add path
run('setup.m');
datadir = fullfile(basedir,'data');
filename = 'ABIDE_controlData_7Sites';
load(fullfile(datadir,filename));
load(fullfile(datadir,'cc200Map2Communities'))
max_recursion_depth(1024)

## Exercise - Does global signal induced nuisance covariance matrix show site effects?

1. Estimate nuisance covariance matrix using mean ROI signal as nuisance covariate
2. Apply site effect statistic on nuisance matrices

In [2]:
disp('Fields in dataset')
fieldnames(data{1})

disp('signals contains subjects x brain regions x time-series')
disp(size(data{1}.signals))

namefield = 'dataName';
results = {};
for dataset=[1:7]
    disp(data{dataset}.(namefield))
    nsubjects = size(data{dataset}.signals,1);
    denoised_corr = []; 
    nuisance_corr = []; 
    nsr = [];
    for subjectno=1:nsubjects
        X = squeeze(data{dataset}.signals(subjectno,:,:))';
        Y = mean(X,2);
        [Sigma output] = covariance.sample_conditional_correlation(X, ...
                                    struct('verbose',false), ...	
                                    'nuisance',Y);
        denoised_corr(subjectno,:,:) = Sigma;
        nuisance_corr(subjectno,:,:) = output.nCov; 
        nsr(subjectno) = output.NSR;
        if(mod(subjectno,10)==0)
            disp(['Subject ...' num2str(subjectno)]);
            disp('Nuisance to Signal Ratio')
            disp(real(output.NSR))
        end
    end
    results{dataset}.denoised_corr = denoised_corr; 
    results{dataset}.nuisance_corr = nuisance_corr;
    results{dataset}.nsr = nsr;
end

Fields in dataset


ans = 


{


  [1,1] = signals


  [2,1] = subIDs


...

Nuisance to Signal Ratio


 0.23785


Subject ...20


Nuisance to Signal Ratio


 0.16400


In [3]:
concat_corr = @(corr1,corr2)(real(cat(1,corr1,corr2)));

sitecluster = []; 
dataset = [1:5];
currtotal = 0;
denoised_corr = [];
nuisance_corr = [];
nsr = [];
for ii=1:length(dataset)
    switch dataset(ii)
      case 1
      clusterval = 1;
      case 2
      clusterval = 1;
      case 3
      clusterval = 3;
      case 4
      clusterval = 4;
      case 5
      clusterval = 5;
      case 6
      clusterval = 6;
      case 7
      clusterval = 7;
    end
    sitecluster(currtotal+1:currtotal+size(data{dataset(ii)}.signals,1)) = clusterval;
    currtotal = currtotal+size(data{dataset(ii)}.signals,1);
  
    denoised_corr = concat_corr(denoised_corr,results{dataset(ii)}.('denoised_corr'));
    nuisance_corr = concat_corr(nuisance_corr,results{dataset(ii)}.('nuisance_corr'));
    nsr = concat_corr(nsr,results{dataset(ii)}.('nsr')'); 
end
  
reshape_corr = @(corr)(reshape(corr,[size(corr,1) size(corr,2)*size(corr,3)]));
denoised_corr = reshape_corr(denoised_corr);
nuisance_corr = reshape_corr(nuisance_corr);
figure;
subplot(1,2,1); imagesc(corr(denoised_corr')); colorbar; axis equal image;
subplot(1,2,2); imagesc(corr(nuisance_corr')); colorbar; axis equal image;


% Apply site detection statistic
[hom_com sep_com mw_com] = compareWithinAndBetweenGroupsSim(corr((denoised_corr+nuisance_corr)'), sitecluster)
communityAgrmntScore=mw_com.stats.zval;

[hom_com sep_com mw_com] = compareWithinAndBetweenGroupsSim(corr(denoised_corr'), sitecluster)
communityAgrmntScore=mw_com.stats.zval;

% Apply site detection statistic
[hom_com sep_com mw_com] = compareWithinAndBetweenGroupsSim(corr(nuisance_corr'), sitecluster)
communityAgrmntScore=mw_com.stats.zval;


MANN-WHITNEY-WILCOXON TEST


---------------------------------------------------------------------------


				Group 1		Group 2


numerosity			14447		6874


Sum of Ranks (W)		145565242.0		81737939.0


...

      scalar structure containing the fields:


        zval =  14.637


        ranksum =  147870802


    p = 0


    h =  1


In [4]:
% Plot NSR by Site
figure; 
plot(sitecluster,nsr,'o'); 
xlim([-.5 length(data)+1]); 
xlabel('Site'); 
ylabel('Nuisance to Signal Ratio')

### Reduction in site effects using conditional correlation

**Nuisance covariates used are principal components from shared within-site variation**

In [7]:
results = analyze_abide_siteeffects('within_site')

ans = 0


Site sitedata_LEUVEN_1, Processing Subject 5 ...


Site sitedata_LEUVEN_1, Processing Subject 10 ...


Site sitedata_LEUVEN_1, Processing Subject 15 ...


Site sitedata_LEUVEN_2, Processing Subject 5 ...


...

    Y


    X_perpY


    denoised


    nuisance


    nsr


#### Exercise: In some sites the nuisance variation in global signal sometimes contains desired BOLD signal. Find which one?

Modify `line 33` to change the set of sites used in the analysis and then re-run `results = analyze_abide_siteeffects('within_site')`
You will find that one of the sites is very different from the other sites. Why?